# https://medium.com/coinmonks/word-level-lstm-text-generator-creating-automatic-song-lyrics-with-neural-networks-b8a1617104fb
### get the pos in order, what occurs
### amd then also get words for those pos and fill in the blanks to make a new one


### train with sequences of words and then guess the target next word

### The idea is that after many epochs the RNN will learn “the style” of how the corpus is written, trying to adjust the weights of the network to predict the next word given a sequence of the N previous words.


In [2]:
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.utils import to_categorical
from  tensorflow.keras.preprocessing import text_dataset_from_directory
import tensorflow
# from tensorflow.keras.metrics import 


## Reading the corpus, split into words
### The first step is to read the corpus and split it into words.

## Note the call to .replace(‘\n’, ‘ \n ‘, this is because we want the newline as a word. The idea behind this is that we are also leaving to the network the decision on when to start a new line (after several words). After this, text_in_words is a big array containing all the corpus, word by word.

# https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92

# https://medium.com/@enriqueav/update-automatic-song-lyrics-creator-with-word-embeddings-e30de94db8d1

### A trained layer converts the words in the vocabulary in a way that the ones with similar use will be close in their vectorial representation. This means, for instance, that words representing colors: “white”, “red”, “yellow”, etc will be close among them.

### https://github.com/ApurbaSengupta/Lyrics-Generation-using-BERT/blob/master/code/reconstruction.ipynb


one idea is to take all of the sentences from a given genre and then use word2vec to find simmilar senteacnes and swamp them around 
https://datascience.stackexchange.com/questions/23969/sentence-similarity-prediction

another is to switch out word for word, start at first word guess the second then third and so on (replace all words with simmilar words)

another is to remove certain words and then replcae those with a guess based on the previous context

https://datascience.stackexchange.com/questions/9785/predicting-a-word-using-word2vec-model

use word2vec to o thro


## imports

In [3]:
import gensim.models
import os
from gensim.models import Phrases
import logging
import re, string
import nltk
import io
# import gensim.downloader as api
# # googleWordVec = api.load('word2vec-google-news-300')
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# method 1: keep sentance structure (leave stopwords) but compute new words for each using missing word (word2vec) - gram

### import the data

In [4]:
def importData(directory: str) -> list:
    with open(directory) as f:
        return f.read()

def getAllFilesInFolder(dir: str) -> list:
    return os.listdir(dir)

def cleanText(text: str):
    splitText = text.split(" ")
    return splitText


In [5]:
countrySongs = []
for file in getAllFilesInFolder("./data/Country"):
    filePath = "./data/Country/{}".format(file)
    if "DS_Store" not in filePath:
        countrySongs.append(importData(filePath))

len(countrySongs)


100

In [6]:
#  each sentance should be  list of words
# model = gensim.models.Word2Vec(sentences=countrySongs[0][0])


# # Train a bigram detector.
# bigram_transformer = Phrases(allCountryLyrics)
# bigram_transformer.export_phrases()
# # bigram_transformer.analyze_sentence  USE this to make new songs 1

# xy = allCountryLyrics[0]
# bigram_transformer[allCountryLyrics]

# # Apply the trained MWE detector to a corpus, using the result to train a Word2vec model.
# countryWordVecModel = gensim.models.Word2Vec(allCountryLyrics, min_count=1)

In [7]:

# newSongLine = []
# for word in allCountryLyrics[0]:
#     print(word)
#     x = countryWordVecModel.wv.most_similar(word)[0]
#     newSongLine.append(x)


In [8]:

# countryBigramModel.wv.most_similar("the")
# googleWordVec.most_similar("a")


# for word in newSongLine:
#     # print(word[0])
#     print(googleWordVec.most_similar(word[0])[0])

## doc 2 vec countrySongs



In [9]:

# source text -> allCountryLyrics
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(countrySongs)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# Transforms each text in texts to a sequence of integers.
encoded = tokenizer.texts_to_sequences(countrySongs)
len(encoded)


Vocabulary Size: 2803


100

In [10]:

# create word -> word sequences 
# get sequences of words together, so given "a b c d"
#  you should give [a,b], [b,c], [c,d] all the adjacent pairs
sequences = []
for i in range(len(encoded)):
	for j in range(2, len(encoded[i])):
		sequence = encoded[i][j-2:j+1]
		sequences.append(sequence)
		
		
print('Total Sequences: %d' % len(sequences))
sequences

Total Sequences: 25367


[[2, 458, 14],
 [458, 14, 516],
 [14, 516, 2],
 [516, 2, 32],
 [2, 32, 408],
 [32, 408, 3],
 [408, 3, 5],
 [3, 5, 459],
 [5, 459, 460],
 [459, 460, 1439],
 [460, 1439, 34],
 [1439, 34, 1],
 [34, 1, 161],
 [1, 161, 118],
 [161, 118, 169],
 [118, 169, 18],
 [169, 18, 5],
 [18, 5, 43],
 [5, 43, 320],
 [43, 320, 112],
 [320, 112, 1440],
 [112, 1440, 3],
 [1440, 3, 89],
 [3, 89, 3],
 [89, 3, 169],
 [3, 169, 94],
 [169, 94, 38],
 [94, 38, 110],
 [38, 110, 4],
 [110, 4, 95],
 [4, 95, 110],
 [95, 110, 156],
 [110, 156, 95],
 [156, 95, 58],
 [95, 58, 2],
 [58, 2, 458],
 [2, 458, 14],
 [458, 14, 516],
 [14, 516, 2],
 [516, 2, 32],
 [2, 32, 408],
 [32, 408, 3],
 [408, 3, 5],
 [3, 5, 459],
 [5, 459, 1104],
 [459, 1104, 73],
 [1104, 73, 261],
 [73, 261, 3],
 [261, 3, 178],
 [3, 178, 29],
 [178, 29, 188],
 [29, 188, 1105],
 [188, 1105, 1441],
 [1105, 1441, 19],
 [1441, 19, 3],
 [19, 3, 23],
 [3, 23, 116],
 [23, 116, 611],
 [116, 611, 1442],
 [611, 1442, 12],
 [1442, 12, 1443],
 [12, 1443, 11],
 [144

In [32]:
import numpy
from keras.preprocessing.sequence import pad_sequences


# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)

# create word -> word sequences 
# get sequences of words together, so given "a b c d"
#  you should give [a,b,c], [b,c, d]... all the adjacent triples
sequences = numpy.array(sequences)
testData, testLabels = sequences[:,:-1],sequences[:,-1]
# one hot encode outputs -> Converts a class vector (integers) to binary class matrix.
testLabels= to_categorical(testLabels, num_classes=vocab_size)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(testData, testLabels, epochs=100, verbose=2)
# evaluate
# print(generate_seq(model, tokenizer, 'Jack', 6))

Max Sequence Length: 3
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 10)             28030     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_2 (Dense)              (None, 2803)              142953    
Total params: 183,183
Trainable params: 183,183
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
793/793 - 4s - loss: 6.5112 - accuracy: 0.0386
Epoch 2/100
793/793 - 2s - loss: 6.1086 - accuracy: 0.0403
Epoch 3/100
793/793 - 2s - loss: 6.0099 - accuracy: 0.0406
Epoch 4/100
793/793 - 2s - loss: 5.9482 - accuracy: 0.0427
Epoch 5/100
793/793 - 2s - loss: 5.8736 - accuracy: 0.0489
Epoch 6/100
793/793 - 2s - loss: 5.7623 - accuracy: 0.0

In [33]:
result = ""
encodedExample = tokenizer.texts_to_sequences(["the"])[0]
encodedExample = numpy.array(encodedExample)
yhat = numpy.argmax(model.predict(encodedExample), axis=-1)
yhat

out_word = ''
for word, index in tokenizer.word_index.items():
    if index == yhat:
        out_word = word
        break
# append to input
in_text, result = out_word, result + ' ' + out_word
print(in_text)
print(out_word)
print(result)

lord
lord
 lord


In [34]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = numpy.argmax(model.predict(encoded), axis=-1)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

In [42]:
# print(generate_seq(model, tokenizer, "I", 6))
print(generate_seq(model, tokenizer, max_length-1, 'my queen', 7))

#After the model is fit, we test it by passing it a given word from the vocabulary and having the model predict the next word. 
# Here we pass in ‘Jack‘ by encoding it and calling model.predict_classes() to get the integer output for the predicted word. 
# This is then looked up in the vocabulary mapping to give the associated word.

my queen you're the one i want to thank


In [ ]:
# they went precious times there i know who holds
# who holds my handev'ry step is my wish i
# wish i was born and raisedthe place where i
# where i grew content to be my queen
# my queen you're the one i want to thank

NameError: name 'tokenizer' is not defined

## eval with BLEU score
https://towardsdatascience.com/how-to-evaluate-text-generation-models-metrics-for-automatic-evaluation-of-nlp-models-e1c251b04ec1
